In [10]:
import os
from dotenv import load_dotenv
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime
import matplotlib.pyplot as plt

physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
# try:
#   tf.config.experimental.set_memory_growth(physical_devices[0], True)
# except:
#   # Invalid device or cannot modify virtual devices once initialized.
#   pass

load_dotenv(override=True)

DATA_PATH = os.getenv('DATA_PATH')
print(DATA_PATH)

# Choose subontology (CCO, MFO or BPO)
SO = 'BPO'

Num GPUs Available:  1
/mnt/e/ML/cafa-5-protein-function-prediction


## Reading fasta, obo and tsv files

In [2]:
from Bio import SeqIO

sequences = [rec.seq for rec in SeqIO.parse(os.path.join(DATA_PATH, "Train/train_sequences.fasta"),"fasta")]
ids = [rec.id for rec in SeqIO.parse(os.path.join(DATA_PATH, "Train/train_sequences.fasta"),"fasta")]

In [3]:
import networkx
import obonet

# Read the taxrank ontology
url = os.path.join(DATA_PATH, "Train/go-basic.obo")
graph = obonet.read_obo(url)



In [4]:
df = pd.read_csv(os.path.join(DATA_PATH, "Train/train_terms.tsv"), sep='\t')

dfSO = df.loc[df["aspect"]==SO]
uniqueTerms = dfSO["term"].unique()
termsArr = list(dfSO["term"].to_numpy())

uniqueTermsDict={}
for i,el in enumerate(uniqueTerms):
    uniqueTermsDict[el] = i
    
print(dfSO.shape)
df=dfSO

df.set_index("EntryID", inplace=True)

(3497732, 3)


In [5]:
testID = df.index.to_list()[0]

## GO analysis

In [6]:
item_counts = df["term"].value_counts()

In [7]:
id_to_name = {id_: data.get('name') for id_, data in graph.nodes(data=True)}
name_to_id = {data['name']: id_ for id_, data in graph.nodes(data=True) if 'name' in data}

## Label encoding

The task is a multilabel classification: The output has several possible targets (Gene Ontologies) but each can only be 1 (existing) or 0 (non existing)

Extract label weights from IA

In [8]:
dfIa = pd.read_csv(os.path.join(DATA_PATH, "IA.txt"), sep='\t', header=None)

dfIa.set_index(0, inplace=True)

labelWeights=[]
allIndices = dfIa.index.tolist()

# if SO == "BPO":
#     item_counts=item_counts[0:10]
#     # item_counts=item_counts[0:len(item_counts)//2]

notFound=0
for go in item_counts.index.to_list():
    if go in allIndices:
        labelWeights.append(dfIa.loc[go].to_numpy()[0])
    else:
        notFound += 1
        labelWeights.append(0)

print("Not found GOs: {} (set to 0)".format(notFound))

Not found GOs: 0 (set to 0)


In [31]:
from sklearn.preprocessing import MultiLabelBinarizer
import pickle

topGOs= item_counts
topGOs=topGOs.index.to_list()

threshold=2
labelWeights=np.array(labelWeights)
selection = labelWeights>threshold
topGOs=np.array(topGOs)[selection]


mlb = MultiLabelBinarizer()
mlb.fit([topGOs])

dftest=df.loc[testID]
indices = dftest["term"].to_numpy()
print(indices)
print(mlb.transform([indices]))
print(len(mlb.classes_))

with open(os.path.join(DATA_PATH,'MLB_'+SO+'.pkl'), 'wb') as f:
    pickle.dump(mlb, f)

['GO:0008152' 'GO:0034655' 'GO:0072523' 'GO:0044270' 'GO:0006753'
 'GO:1901292' 'GO:0044237' 'GO:1901360' 'GO:0008150' 'GO:1901564'
 'GO:1901565' 'GO:0009117' 'GO:0006139' 'GO:0044281' 'GO:0046496'
 'GO:0019362' 'GO:0046483' 'GO:0055086' 'GO:0044248' 'GO:0019439'
 'GO:0019637' 'GO:0006807' 'GO:0019677' 'GO:1901361' 'GO:0006163'
 'GO:0046700' 'GO:0009987' 'GO:0006725' 'GO:0006796' 'GO:0034641'
 'GO:0072521' 'GO:0071704' 'GO:0019364' 'GO:1901575' 'GO:0072526'
 'GO:0046434' 'GO:0009166' 'GO:0072524' 'GO:0006195' 'GO:0009056'
 'GO:0044238' 'GO:0006793' 'GO:0019674']
[[0 0 0 ... 0 0 0]]
6355


/home/manuel/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['GO:0006139', 'GO:0006163', 'GO:0006195', 'GO:0006725', 'GO:0006753', 'GO:0006796', 'GO:0006807', 'GO:0008150', 'GO:0008152', 'GO:0009117', 'GO:0009166', 'GO:0009987', 'GO:0019362', 'GO:0019364', 'GO:0019439', 'GO:0019637', 'GO:0019674', 'GO:0019677', 'GO:0034641', 'GO:0034655', 'GO:0044237', 'GO:0044238', 'GO:0044248', 'GO:0044270', 'GO:0046434', 'GO:0046483', 'GO:0046496', 'GO:0046700', 'GO:0055086', 'GO:0071704', 'GO:0072521', 'GO:0072523', 'GO:0072526', 'GO:1901292', 'GO:1901360', 'GO:1901361', 'GO:1901564', 'GO:1901565', 'GO:1901575'] will be ignored
  warnings.warn(


## Amino acids encoding

In [10]:
aminos_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'X']

In [11]:
aa_dict = {'A': 1, 'B':24, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'O': 21, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'U': 22, 'V': 18, 'W': 19, 'Y': 20, 'X':30, 'Z':23}

## Build Dataset

In [12]:
seqLengths = [len(seq) for seq in sequences]
maxLen = max(seqLengths)
print("The max. length of the sequences is {}".format(maxLen))

The max. length of the sequences is 35375


In [13]:

dfAll=pd.read_csv(os.path.join(DATA_PATH, "Train/train_terms.tsv"), sep='\t')

soEntries = dfAll.loc[dfAll["aspect"]==SO]
soEntryIds = soEntries["EntryID"].unique()

# print(len(seqEntries))
print(soEntryIds)

# SoSequences = []
# for entry in soEntryIds:
#     SoSequences.append(sequences[ids.index(entry)])

# print(len(SoSequences))
dfAll.set_index("EntryID", inplace=True)

['A0A009IHW8' 'A0A021WW32' 'A0A023FFD0' ... 'X5L1L5' 'X5L565' 'X5M5N0']


In [14]:
import warnings
from itertools import product
from tqdm import tqdm
import os

TRAIN_VAL_SPLIT = 0.7
k = 3
n=100
maxFreqVectorLen = 5000//n

allAA = list(aa_dict.keys())
allAA.sort()
allCombinations= list(product(*(allAA for i in range(k))))
allCombinations=np.array([''.join(el) for el in allCombinations])

positionDict = dict(zip(allCombinations, np.arange(0,allCombinations.size).T))

xSize = allCombinations.shape[0]

#Use numpy vectorize to speed up the mapping (hopefully)
mapping = lambda x: aa_dict[x]
vectMapping = np.vectorize(mapping)

# Shuffle the data
import random
random.seed(516213)
c = list(zip(sequences, ids))
random.shuffle(c)
sequencesShuffle, idsShuffle = zip(*c)


#Train Validation Split
split = int(np.floor(len(sequencesShuffle)*TRAIN_VAL_SPLIT))
print(split)
trainSeq = sequencesShuffle[0:split]
valSeq = sequencesShuffle[split+1:]
trainIds = idsShuffle[0:split]
valIds = idsShuffle[split+1:]


def generator():
  for i,seq in enumerate(trainSeq):
      entryId = trainIds[i]
      
      if entryId in soEntryIds:
        labelData = df.loc[entryId]
        # indices = labelData["termToken"].to_numpy()
        indices = labelData["term"].to_numpy()
      else:
        indices=[]

      with warnings.catch_warnings():
          #supress the warnings for unknown classes
          warnings.simplefilter("ignore")
          y = mlb.transform([indices])
      
      seqLen= len(seq)
      freqVectorSeq=[]
      for subIdx in np.arange(0, seqLen//n+1):
        subSeq = seq[subIdx*n:(subIdx+1)*n]
        if len(subSeq)<3:
          continue
        kmers = [subSeq[i:i+k] if i < len(subSeq)-(k-1) else 0 for i,el in enumerate(subSeq)]
        kmers = kmers[0:-(k-1)]
        kmers = [str(el) for el in kmers]
        values, counts = np.unique(kmers, return_counts=True)
        freqVector=np.zeros(allCombinations.shape)
        for j,v in enumerate(values):
          freqVector[positionDict[v]] = counts[j]
        freqVectorSeq.append(freqVector)

      #Pad or truncate the frequency vector sequence
      if len(freqVectorSeq)>maxFreqVectorLen:
          freqVectorSeq=np.array(freqVectorSeq[0:maxFreqVectorLen])
      elif len(freqVectorSeq)<maxFreqVectorLen:
          diffLen = maxFreqVectorLen - len(freqVectorSeq)
          padding = np.zeros((diffLen,xSize))
          freqVectorSeq = np.append(np.array(freqVectorSeq), padding, axis=0)
      else:
         freqVectorSeq= np.array(freqVectorSeq)
      yield (freqVectorSeq,y[0])


def generatorVal():
  for i,seq in enumerate(valSeq):
      entryId = valIds[i]
      if entryId in soEntryIds:
        labelData = df.loc[entryId]
        # indices = labelData["termToken"].to_numpy()
        indices = labelData["term"].to_numpy()
      else:
        indices=[]

      with warnings.catch_warnings():
          #supress the warnings for unknown classes
          warnings.simplefilter("ignore")
          y = mlb.transform([indices])
      
      seqLen= len(seq)
      freqVectorSeq=[]
      for subIdx in np.arange(0, seqLen//n+1):
        subSeq = seq[subIdx*n:(subIdx+1)*n]
        if len(subSeq)<3:
          continue
        kmers = [subSeq[i:i+k] if i < len(subSeq)-(k-1) else 0 for i,el in enumerate(subSeq)]
        kmers = kmers[0:-(k-1)]
        kmers = [str(el) for el in kmers]
        values, counts = np.unique(kmers, return_counts=True)
        freqVector=np.zeros(allCombinations.shape)
        for j,v in enumerate(values):
          freqVector[positionDict[v]] = counts[j]
        freqVectorSeq.append(freqVector)
        
      #Pad or truncate the frequency vector sequence
      if len(freqVectorSeq)>maxFreqVectorLen:
          freqVectorSeq=np.array(freqVectorSeq[0:maxFreqVectorLen])
      elif len(freqVectorSeq)<maxFreqVectorLen:
          diffLen = maxFreqVectorLen - len(freqVectorSeq)
          padding = np.zeros((diffLen,xSize))
          freqVectorSeq = np.append(np.array(freqVectorSeq), padding, axis=0)
      else:
         freqVectorSeq= np.array(freqVectorSeq)
      yield (freqVectorSeq,y[0])
        

99572


In [15]:
g = generator()
test = next(g)
print("The first sample: \n{}\n{}".format(test[0].shape, test[0]))
print("The first sample has {} classes".format(np.count_nonzero(test[1])))

The first sample: 
(50, 15625)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
The first sample has 0 classes


## Tensorflow Classification

In [16]:


dataset = tf.data.Dataset.from_generator(generator, output_signature=(
         tf.TensorSpec(shape=(None,xSize), dtype=tf.int32),
         tf.TensorSpec(shape=(len(mlb.classes_),), dtype=tf.int32)))
print(list(dataset.take(1)))

datasetVal = tf.data.Dataset.from_generator(generatorVal, output_signature=(
         tf.TensorSpec(shape=(None,xSize), dtype=tf.int32),
         tf.TensorSpec(shape=(len(mlb.classes_),), dtype=tf.int32)))

2023-06-29 12:08:23.927785: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-29 12:08:23.928125: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-29 12:08:23.928420: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-29 12:08:24.377422: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-29 12:08:24.377825: I tensorflow/compile

[(<tf.Tensor: shape=(50, 15625), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, <tf.Tensor: shape=(21285,), dtype=int32, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=int32)>)]


2023-06-29 12:08:24.684931: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [17]:
%load_ext tensorboard


In [18]:

from tensorflow.keras import layers

VOCAB_SIZE=len(aa_dict)
EMBED_DIM=500
OUT_SEQ_LENGTH=10

def createRnnModel():
    inputs = tf.keras.Input(shape=(maxFreqVectorLen, xSize,))
    # x = tf.keras.layers.Masking(0)(inputs)
    # x=layers.Embedding(VOCAB_SIZE, EMBED_DIM, name="embedding")(inputs)

    # x = layers.RepeatVector(OUT_SEQ_LENGTH)(x)
    x=layers.BatchNormalization()(inputs)
    x = layers.Bidirectional(layers.LSTM(10, return_sequences=True))(x)
    x=layers.BatchNormalization()(x)
    x = layers.Bidirectional(layers.LSTM(10))(x)
    x=layers.BatchNormalization()(x)
    # x = layers.LSTM(64)(x)
    x = layers.Dense(128)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(128)(x)
    outputs=layers.Dense(len(mlb.classes_), activation=tf.keras.activations.sigmoid)(x)
    # outputs=layers.Softmax()(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="embedRnnModel")

model = createRnnModel()

model.summary()

2023-06-29 12:08:24.897563: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-29 12:08:24.898655: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-29 12:08:24.899410: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "embedRnnModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 15625)]       0         
                                                                 
 batch_normalization (BatchN  (None, 50, 15625)        62500     
 ormalization)                                                   
                                                                 
 bidirectional (Bidirectiona  (None, 50, 20)           1250880   
 l)                                                              
                                                                 
 batch_normalization_1 (Batc  (None, 50, 20)           80        
 hNormalization)                                                 
                                                                 
 bidirectional_1 (Bidirectio  (None, 20)               2480      
 nal)                                                

2023-06-29 12:08:25.127337: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-29 12:08:25.128385: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-29 12:08:25.129221: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [19]:
import matplotlib.pyplot as plt
#Learning rate schedule
initial_learning_rate = 0.001
decaySteps=5000
lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate, first_decay_steps=decaySteps,
                                                                t_mul=2.0, m_mul=0.7)
# lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate, decay_steps=decaySteps, alpha=0.01)
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,decay_steps=decaySteps,decay_rate=0.9,staircase=False)
step = np.linspace(0,decaySteps*3)
lr = lr_schedule(step)
# plt.figure(figsize = (8,6))
# plt.yscale("log")
# plt.plot(step, lr)
# plt.ylim([0,max(plt.ylim())])
# plt.xlabel('step')
# _ = plt.ylabel('Learning Rate')

In [20]:
from utils import *

In [21]:
# import tensorflow_addons as tfa

BATCH_SIZE=32
LOG_INTERVAL=20
epochs = 15
saveModel=False


log_dir = "./logs/"+model.name+"/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_"+SO
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,
                                                      write_graph=True, update_freq=5)


summary_writer = tf.summary.create_file_writer(log_dir)

# Instantiate an optimizer .
# optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Instantiate a loss function.
# loss_fn = tf.keras.losses.BinaryCrossentropy()
# loss_fn = WeightedBinaryCE(np.ones(len(mlb.classes_)))
# loss_fn = WeightedBinaryCE(labelWeights)
loss_fn = WeightedComboLoss(labelWeights, alpha=0.5, beta=0.5)

train_acc_metric = WeightedAccuracy(classWeights=labelWeights)
train_f1_metric = WeightedF1(classWeights=labelWeights, threshold=0.5)
train_prec = WeightedPrecision(classWeights=labelWeights)
train_rec = WeightedRecall(classWeights=labelWeights)

val_acc_metric = WeightedAccuracy(classWeights=labelWeights)
val_f1_metric = WeightedF1(classWeights=labelWeights, threshold=0.5)
val_prec = WeightedPrecision(classWeights=labelWeights)
val_rec = WeightedRecall(classWeights=labelWeights)

batchedDataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
batchedDatasetVal = datasetVal.batch(BATCH_SIZE, drop_remainder=False)

# batchedDataset = batchedDataset.cache(os.path.join(DATA_PATH, "datasetCache"+SO))
# batchedDatasetVal = batchedDatasetVal.cache(os.path.join(DATA_PATH, "datasetCacheVal"+SO))

@tf.function()
def trainStep(x_batch_train, y_batch_train):
    with tf.GradientTape() as tape:

        probs = model(x_batch_train, training=True) 
        loss_value = loss_fn(y_batch_train, probs)

    grads = tape.gradient(loss_value, model.trainable_weights)
    #Gradient clipping
    # grads = [tf.clip_by_norm(g, 2.0) for g in grads]

    train_acc_metric.update_state(y_batch_train, probs)
    train_f1_metric.update_state(y_batch_train, probs)
    train_prec.update_state(y_batch_train, probs)
    train_rec.update_state(y_batch_train, probs)

    optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
    return loss_value

@tf.function()
def valStep(x_batch_val, y_batch_val):
    valProbs = model(x_batch_val, training=False)
    # Update val metrics
    val_acc_metric.update_state(y_batch_val, valProbs)
    val_f1_metric.update_state(y_batch_val, valProbs)
    val_prec.update_state(y_batch_val, valProbs)
    val_rec.update_state(y_batch_val, valProbs)

maxStep=0

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(batchedDataset):

        loss_value =trainStep(x_batch_train,y_batch_train)

        # Log 
        if step % LOG_INTERVAL == 0:
            template = 'Epoch {}/Step {}, Loss: {:.5f}, Accuracy: {:.5f}, F1: {:.4f}, Prec: {:.4f}, Rec: {:.4f} lr: {:.5f}'
            print(template.format(epoch+1, step,loss_value.numpy(), 
                                    train_acc_metric.result(),train_f1_metric.result(),
                                    train_prec.result(), train_rec.result(), optimizer.learning_rate.numpy()))
            
            with summary_writer.as_default():
                tf.summary.scalar('loss', loss_value, step=maxStep*epoch+step)
                tf.summary.scalar('accuracy', train_acc_metric.result(), step=maxStep*epoch+step)
                tf.summary.scalar('f1', train_f1_metric.result(), step=maxStep*epoch+step)
                tf.summary.scalar('prec', train_prec.result(), step=maxStep*epoch+step)
                tf.summary.scalar('rec', train_rec.result(), step=maxStep*epoch+step)
                tf.summary.scalar('learning rate', optimizer.learning_rate.numpy(), step=maxStep*epoch+step)
                summary_writer.flush()

    
    train_acc_metric.reset_states()
    train_f1_metric.reset_states()
    train_prec.reset_states()
    train_rec.reset_states()

    maxStep=step

    print("Epoch finished. Start validation")
    for x_batch_val, y_batch_val in batchedDatasetVal:
        valStep(x_batch_val, y_batch_val)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    val_f1 = val_f1_metric.result()
    val_f1_metric.reset_states()
    val_precision = val_prec.result()
    val_prec.reset_states()
    val_recall = val_rec.result()
    val_rec.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Validation f1: %.4f" % (float(val_f1),))
    print("Validation precision: %.4f" % (float(val_precision),))
    print("Validation recall: %.4f" % (float(val_recall),))
    with summary_writer.as_default():
        tf.summary.scalar('valAcc', float(val_acc), step=epoch)
        tf.summary.scalar('valF1', float(val_f1), step=epoch)
        tf.summary.scalar('valPrecision', float(val_precision), step=epoch)
        tf.summary.scalar('valRecall', float(val_recall), step=epoch)
        summary_writer.flush()
    if saveModel:
      model.save(os.path.join(DATA_PATH, "model_"+SO+"_epoch_{}_valF1Score{:.3f}".format(epoch, float(val_f1))))


Start of epoch 1


2023-06-29 12:08:25.509151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-29 12:08:25.978581: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-29 12:08:25.980344: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

Epoch 1/Step 0, Loss: 0.29653, Accuracy: 0.50239, F1: 0.0020, Prec: 0.3520, Rec: 0.0010 lr: 0.00100
Epoch 1/Step 20, Loss: 0.14601, Accuracy: 0.90894, F1: 0.0261, Prec: 0.1211, Rec: 0.0250 lr: 0.00100
Epoch 1/Step 40, Loss: 0.00517, Accuracy: 0.95270, F1: 0.0516, Prec: 0.0899, Rec: 0.0788 lr: 0.00100
Epoch 1/Step 60, Loss: -0.00430, Accuracy: 0.96773, F1: 0.0706, Prec: 0.0899, Rec: 0.1034 lr: 0.00100
Epoch 1/Step 80, Loss: -0.04190, Accuracy: 0.97512, F1: 0.1024, Prec: 0.1223, Rec: 0.1237 lr: 0.00100
Epoch 1/Step 100, Loss: -0.06933, Accuracy: 0.97944, F1: 0.1181, Prec: 0.1465, Rec: 0.1290 lr: 0.00100
Epoch 1/Step 120, Loss: -0.09712, Accuracy: 0.98247, F1: 0.1298, Prec: 0.1580, Rec: 0.1359 lr: 0.00100
Epoch 1/Step 140, Loss: -0.08597, Accuracy: 0.98465, F1: 0.1379, Prec: 0.1647, Rec: 0.1415 lr: 0.00100
Epoch 1/Step 160, Loss: -0.08404, Accuracy: 0.98633, F1: 0.1451, Prec: 0.1672, Rec: 0.1504 lr: 0.00100
Epoch 1/Step 180, Loss: -0.08279, Accuracy: 0.98764, F1: 0.1512, Prec: 0.1693, Rec

2023-06-29 12:17:09.454918: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-29 12:17:09.455998: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-29 12:17:09.456713: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch finished. Start validation


2023-06-29 12:17:10.877195: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-29 12:17:11.277046: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-29 12:17:11.278807: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

Validation acc: 0.9981
Validation f1: 0.2047
Validation precision: 0.1963
Validation recall: 0.2190

Start of epoch 2
Epoch 2/Step 0, Loss: -0.11104, Accuracy: 0.99816, F1: 0.2301, Prec: 0.2250, Rec: 0.2354 lr: 0.00100
Epoch 2/Step 20, Loss: -0.10803, Accuracy: 0.99807, F1: 0.2232, Prec: 0.2086, Rec: 0.2429 lr: 0.00100
Epoch 2/Step 40, Loss: -0.10870, Accuracy: 0.99810, F1: 0.2208, Prec: 0.2097, Rec: 0.2370 lr: 0.00100
Epoch 2/Step 60, Loss: -0.10323, Accuracy: 0.99814, F1: 0.2153, Prec: 0.2063, Rec: 0.2293 lr: 0.00100
Epoch 2/Step 80, Loss: -0.08895, Accuracy: 0.99816, F1: 0.2153, Prec: 0.2035, Rec: 0.2337 lr: 0.00100
Epoch 2/Step 100, Loss: -0.11890, Accuracy: 0.99818, F1: 0.2146, Prec: 0.2013, Rec: 0.2350 lr: 0.00100
Epoch 2/Step 120, Loss: -0.12117, Accuracy: 0.99818, F1: 0.2132, Prec: 0.2009, Rec: 0.2322 lr: 0.00100
Epoch 2/Step 140, Loss: -0.09694, Accuracy: 0.99820, F1: 0.2118, Prec: 0.1996, Rec: 0.2304 lr: 0.00100
Epoch 2/Step 160, Loss: -0.10194, Accuracy: 0.99821, F1: 0.2111,

In [ ]:

# model.save(os.path.join(DATA_PATH, "model_"+SO+"_epoch_{}_valf1Score{:.3f}".format(epoch, float(val_f1))))

In [ ]:

probs= model.predict(tf.expand_dims(list(datasetVal.take(32))[10][0], 0))
prediction= [1 if p > 0.5 else 0 for p in probs[0]]
probabilities= probs[probs>0.5]
# classes = np.argwhere(prediction)
print(mlb.inverse_transform(np.array([prediction])))
print(probabilities)